# Tutorial: Zero-shot cross-lingual topic modeling with ZeroShotTM

(last updated 11-01-2021)

In this tutorial, we are going to use our **Zero-Shot Topic Model** to get the topics out of a collections of articles you will upload here. Then, we are going to predict the topics of unseen documents in an unseen language exploiting the multilingual capabilities of Multilingual BERT. 

## Topic Models 

Topic models allow you to discover latent topics in your documents in a completely unsupervised way. Just use your documents and get topics out.

## Contextualized Topic Models

![](https://raw.githubusercontent.com/MilaNLProc/contextualized-topic-models/master/img/logo.png)

What are Contextualized Topic Models? **CTMs** are a family of topic models that combine the expressive power of BERT embeddings with the unsupervised capabilities of topic models to get topics out of documents. In this tutorial, we are going to use the **Zero-Shot Topic Model** version of the Contextualized Topic Models because we want to tackle the problem of cross-lingual topic prediction. 

## Python Package

You can find our package [here](https://github.com/MilaNLProc/contextualized-topic-models).

![https://travis-ci.com/MilaNLProc/contextualized-topic-models](https://travis-ci.com/MilaNLProc/contextualized-topic-models.svg) ![https://pypi.python.org/pypi/contextualized_topic_models](https://img.shields.io/pypi/v/contextualized_topic_models.svg) ![https://pepy.tech/badge/contextualized-topic-models](https://pepy.tech/badge/contextualized-topic-models)




# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# Installing Contextualized Topic Models

First, we install the contextualized topic model library

In [ ]:
%%capture
!pip install contextualized-topic-models
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -U tqdm

## Restart the Notebook

For the changes to take effect, we now need to restart the notebook.

From the Menu:

Runtime → Restart Runtime

# Data

We are going to need some data. You should upload a file with one document per line. We assume you haven't run any preprocessing script.

However, if you want to first test the model without uploading your data, you can simply use the test file I'm putting here

In [ ]:
!wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt

--2021-01-11 21:49:35--  https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6208417 (5.9M) [text/plain]
Saving to: ‘dbpedia_sample_abstract_20k_unprep.txt.1’

dbpedia_sample_abst 100%[===================>]   5.92M  --.-KB/s    in 0.05s   

2021-01-11 21:49:35 (113 MB/s) - ‘dbpedia_sample_abstract_20k_unprep.txt.1’ saved [6208417/6208417]



In [ ]:
!head -n 1 dbpedia_sample_abstract_20k_unprep.txt

The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry


In [ ]:
text_file = "dbpedia_sample_abstract_20k_unprep.txt" # EDIT THIS WITH THE FILE YOU UPLOAD

# Importing what we need

In [ ]:
from contextualized_topic_models.models.ctm import ZeroShotTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
import nltk

## Preprocessing

Why do we use the **preprocessed text** here? We need text without punctuation to build the bag of word. Also, we might want only to have the most frequent words inside the BoW. Too many words might not help.

In [ ]:
nltk.download('stopwords')

documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()]
sp = WhiteSpacePreprocessing(documents, stopwords_language='english')
preprocessed_documents, unpreprocessed_corpus, vocab = sp.preprocess()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
preprocessed_documents[:2]

['mid peninsula highway proposed across peninsula canadian province ontario although highway connecting hamilton fort south international study published ministry',
 'died march american photographer specialized photography operated studio silver spring maryland later lived florida magazine photographer year']

We don't discard the non-preprocessed texts, because we are going to use them as input for obtaining the contextualized document representations. 

Let's pass our files with preprocess and unpreprocessed data to our `TopicModelDataPreparation` object. This object takes care of creating the bag of words for you and of obtaining the contextualized BERT representations of documents. This operation allows us to create our training dataset.

Note: Here we use the contextualized model "distiluse-base-multilingual-cased", because we need a multilingual model for performing cross-lingual predictions later.  

In [ ]:
tp = TopicModelDataPreparation("distiluse-base-multilingual-cased")

training_dataset = tp.create_training_set(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

100%|██████████| 504M/504M [00:28<00:00, 17.8MB/s]


Batches:   0%|          | 0/100 [00:00<?, ?it/s]

Let's check the first ten words of the vocabulary 

In [ ]:
tp.vocab[:10]

['abbreviated',
 'academic',
 'academy',
 'access',
 'according',
 'achieved',
 'acquired',
 'acre',
 'acres',
 'across']

## Training our Zero-Shot Contextualized Topic Model

Finally, we can fit our new topic model. We will ask the model to find 50 topics in our collection (n_component parameter of the CTM object).

In [ ]:
ctm = ZeroShotTM(input_size=len(tp.vocab), bert_input_size=512, n_components=50, num_epochs=15)
ctm.fit(training_dataset) # run the model


Epoch: [15/15]	 Seen Samples: [300000/300000]	Train Loss: 138.64411013183593	Time: 0:00:05.424988: : 15it [01:21,  5.46s/it]


# Topics

After training, now it is the time to look at our topics: we can use the 

```
get_topic_lists
```

function to get the topics. It also accepts a parameter that allows you to select how many words you want to see for each topic.

If you look at the topics, you will see that they all make sense and are representative of a collection of documents that comes from Wikipedia (general knowledge). Notice that the topics are in English, because we trained the model on English documents.

In [ ]:
ctm.get_topic_lists(5)

[['house', 'built', 'located', 'national', 'historic'],
 ['family', 'found', 'species', 'mm', 'moth'],
 ['district', 'village', 'km', 'county', 'west'],
 ['station', 'line', 'railway', 'river', 'near'],
 ['member', 'politician', 'party', 'general', 'political'],
 ['series', 'game', 'film', 'directed', 'written'],
 ['county', 'located', 'united', 'states', 'city'],
 ['university', 'american', 'professor', 'born', 'law'],
 ['war', 'army', 'french', 'professor', 'british'],
 ['century', 'ii', 'war', 'king', 'greek'],
 ['company', 'software', 'based', 'information', 'developed'],
 ['built', 'house', 'building', 'church', 'story'],
 ['province', 'population', 'district', 'municipality', 'region'],
 ['team', 'football', 'league', 'season', 'games'],
 ['school', 'high', 'university', 'college', 'state'],
 ['television', 'film', 'directed', 'published', 'series'],
 ['county', 'states', 'city', 'united', 'located'],
 ['family', 'genus', 'found', 'plant', 'species'],
 ['american', 'football', 'p

### Let's predict the topics of the training documents 

Ok now we can take a document and see which topic has been assigned to it. Results will obviously change with respect to the documents you are using. For example, let's predict the topic of the first preprocessed document that is talking about a peninsula.

In [ ]:
topics_predictions = ctm.get_thetas(training_dataset, n_samples=5) # get all the topic predictions

/usr/local/lib/python3.6/dist-packages/contextualized_topic_models/models/ctm.py:257: DeprecationWarning: Call to `get_thetas` is deprecated and will be removed in version 2, use `get_doc_topic_distribution` instead
  DeprecationWarning)
Sampling: [5/5]: : 5it [00:23,  4.72s/it]


In [ ]:
preprocessed_documents[0] # see the text of our preprocessed document

'mid peninsula highway proposed across peninsula canadian province ontario although highway connecting hamilton fort south international study published ministry'

In [ ]:
import numpy as np
topic_number = np.argmax(topics_predictions[0]) # get the topic id of the first document

In [ ]:
ctm.get_topic_lists(5)[topic_number] #and the topic should be about natural location related things

['station', 'railway', 'line', 'located', 'street']

### Let's predict the topics of the documents in unseen languages 

It's time to take advantage of the power of multilingual BERT. Let's predict the topics of some Italian documents. First, we download the data as before.

In [ ]:
!wget https://raw.githubusercontent.com/vinid/data/master/italian_documents.txt

--2021-01-11 21:53:28--  https://raw.githubusercontent.com/vinid/data/master/italian_documents.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7122 (7.0K) [text/plain]
Saving to: ‘italian_documents.txt’

italian_documents.t 100%[===================>]   6.96K  --.-KB/s    in 0s      

2021-01-11 21:53:28 (33.1 MB/s) - ‘italian_documents.txt’ saved [7122/7122]



In [ ]:
italian_documents = [line.strip() for line in open("italian_documents.txt", 'r').readlines()]
italian_documents

['Fu l\'ispiratore e uno dei fondatori della rivista "Dau al Set" e dell\'omonimo gruppo artistico ed intellettuale catalano (1948). È ritenuto il massimo esponente della poesia visiva non solo della letteratura catalana, ma il pioniere di questo genere in Spagna e uno dei grandi riferimenti internazionali. La totalità della sua opera creativa lo segnala come uno degli autori più prolifici della cultura occidentale contemporanea. Sebbene schierato nell\'avanguardia della poesia della prima metà del XX secolo (esplorò validamente l\'ipnagogia, il surrealismo e il dadaismo), si esercitò alla scrittura di centinaia di sonetti, odi saffiche e sestine liriche dalla totale perfezione formale alla sperimentazione più estrema, nonché migliaia di poemi in forma libera. In vita pubblicò un\'ottantina di libri lasciandone parecchi d\'inediti. Il suo lavoro letterario comprendeva tutti i generi: poesia, prosa, teatro (la sua cosiddetta "poesia scenica", oltre 350 opere), cinema, musica, cabaret, p

There's no need to do preprocess the documents if you want to do zero-shot topic modeling! (The vocabulary obtained from the Italian documents wouldn't match our English vocabulary!) Let's just pass the italian documents as they are (without preprocessing) to our `TopicModelDataPreparation` object and create the testing dataset. 

In [ ]:
testing_dataset = tp.create_test_set(italian_documents) # create dataset for the testset


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Now we are ready to compute the topic predictions for each document. 


In [ ]:
# n_sample how many times to sample the distribution (see the documentation)
italian_topics_predictions = ctm.get_thetas(testing_dataset, n_samples=5) # get all the topic predictions


/usr/local/lib/python3.6/dist-packages/contextualized_topic_models/models/ctm.py:257: DeprecationWarning: Call to `get_thetas` is deprecated and will be removed in version 2, use `get_doc_topic_distribution` instead
  DeprecationWarning)
Sampling: [5/5]: : 5it [00:01,  4.90it/s]


Let's consider the first one that talks about an artist

In [ ]:
italian_documents[0]

'Fu l\'ispiratore e uno dei fondatori della rivista "Dau al Set" e dell\'omonimo gruppo artistico ed intellettuale catalano (1948). È ritenuto il massimo esponente della poesia visiva non solo della letteratura catalana, ma il pioniere di questo genere in Spagna e uno dei grandi riferimenti internazionali. La totalità della sua opera creativa lo segnala come uno degli autori più prolifici della cultura occidentale contemporanea. Sebbene schierato nell\'avanguardia della poesia della prima metà del XX secolo (esplorò validamente l\'ipnagogia, il surrealismo e il dadaismo), si esercitò alla scrittura di centinaia di sonetti, odi saffiche e sestine liriche dalla totale perfezione formale alla sperimentazione più estrema, nonché migliaia di poemi in forma libera. In vita pubblicò un\'ottantina di libri lasciandone parecchi d\'inediti. Il suo lavoro letterario comprendeva tutti i generi: poesia, prosa, teatro (la sua cosiddetta "poesia scenica", oltre 350 opere), cinema, musica, cabaret, pe

As we did before, let's get the index of most likely topic of the first document and then show the topic words to see if the topic's prediction is accurate

In [ ]:
topic_number = np.argmax(italian_topics_predictions[0]) # get the topic id of the first document
ctm.get_topic_lists(10)[topic_number] 

['son',
 'war',
 'de',
 'century',
 'french',
 'ii',
 'daughter',
 'king',
 'father',
 'cross']